In [42]:
import os
from Utils.read_coord import kml_coord
from Utils.sentinal_request import senti_api
from Utils.forest_cover import get_cover
from Utils.forest_cover import ndvi_api
import numpy as np

In [43]:
file_path = input('Enter kml file path: ')
print(file_path)

if os.path.exists(file_path):
    print('The file exists')

    with open(file_path, 'r', encoding='utf-8-sig') as f:
        lines = f.readlines()

        #print(lines)
else:
    print('The specified file does NOT exist')
    
poly_coord = kml_coord(file_path)
print(poly_coord)

img,resol = senti_api(poly_coord)
print(resol)

Enter kml file path: C:\Users\Harith\EngenuityAI\SPE\SPE-development\sentinal\gAMPOLA TEST SITE 2.kml
C:\Users\Harith\EngenuityAI\SPE\SPE-development\sentinal\gAMPOLA TEST SITE 2.kml
The file exists
gAMPOLA TEST SITE 2, None
[[80.54830488  7.09138959]
 [80.54862792  7.0908058 ]
 [80.54983642  7.09009562]
 [80.55092306  7.09024294]
 [80.55188896  7.09040945]
 [80.55227594  7.09060375]
 [80.55291793  7.09075515]
 [80.55346817  7.09067105]
 [80.55432141  7.09055437]
 [80.55468188  7.09055554]
 [80.55514106  7.09091272]
 [80.55538022  7.09175008]
 [80.55549106  7.09276935]
 [80.55517581  7.09363418]
 [80.55438863  7.09398713]
 [80.55357122  7.09426935]
 [80.55309183  7.09425854]
 [80.55259774  7.09410748]
 [80.55215408  7.09387411]
 [80.55163136  7.09373405]
 [80.55100531  7.09343574]
 [80.55023178  7.09319959]
 [80.55022909  7.09321476]
 [80.54910537  7.09293352]
 [80.54821586  7.0922228 ]
 [80.54830488  7.09138959]]
10


In [44]:
red = img[:, :, 2]
nir = img[:, :, 3]

In [45]:
def normalized_diff(b1, b2):
    """Take two n-dimensional numpy arrays and calculate the normalized
    difference.

    Math will be calculated (b1-b2) / (b1 + b2).

    Parameters
    ----------
    b1, b2 : numpy arrays
        Two numpy arrays that will be used to calculate the normalized
        difference. Math will be calculated (b1-b2) / (b1+b2).

    Returns
    ----------
    n_diff : numpy array
        The element-wise result of (b1-b2) / (b1+b2) calculation. Inf values
        are set to nan. Array returned as masked if result includes nan values.

    Examples
    --------
    >>> import numpy as np
    >>> import earthpy.spatial as es
    >>> # Calculate normalized difference vegetation index
    >>> nir_band = np.array([[6, 7, 8, 9, 10], [16, 17, 18, 19, 20]])
    >>> red_band = np.array([[1, 2, 3, 4, 5], [11, 12, 13, 14, 15]])
    >>> ndvi = es.normalized_diff(b1=nir_band, b2=red_band)
    >>> ndvi
    array([[0.71428571, 0.55555556, 0.45454545, 0.38461538, 0.33333333],
           [0.18518519, 0.17241379, 0.16129032, 0.15151515, 0.14285714]])

    >>> # Calculate normalized burn ratio
    >>> nir_band = np.array([[8, 10, 13, 17, 15], [18, 20, 22, 23, 25]])
    >>> swir_band = np.array([[6, 7, 8, 9, 10], [16, 17, 18, 19, 20]])
    >>> nbr = es.normalized_diff(b1=nir_band, b2=swir_band)
    >>> nbr
    array([[0.14285714, 0.17647059, 0.23809524, 0.30769231, 0.2       ],
           [0.05882353, 0.08108108, 0.1       , 0.0952381 , 0.11111111]])
    """
    
    if not (b1.shape == b2.shape):
        raise ValueError("Both arrays should have the same dimensions")
    
    


    # Ignore warning for division by zero
    with np.errstate(divide="ignore",invalid='ignore'):
        
       
        n_diff = (b1 - b2) / (b1 + b2)

    #Set inf values to nan and provide custom warning
    if np.isinf(n_diff).any():
        warnings.warn(
            "Divide by zero produced infinity values that will be replaced "
            "with nan values",
            Warning,
        )
        n_diff[np.isinf(n_diff)] = np.nan

    # Mask invalid values
    if np.isnan(n_diff).any():
        n_diff = np.ma.masked_invalid(n_diff)

    return n_diff


In [46]:
#np.seterr(divide='ignore',invalid='ignore')
arr = normalized_diff(red,nir)

In [37]:
arr


masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=1e+20,
  dtype=float32)

In [47]:
ndvi = ndvi_api(red, nir)
ndvi

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=1e+20,
  dtype=float32)